# Real Time Face Mask Detection
### task 2
#### Sync Machine Learning Internship
##### Abeera Malakar


In [1]:
! pip install -q kaggle

In [2]:
# Acessing direct dataset from kaggle

!rm -r ~/.kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

'rm' is not recognized as an internal or external command,
operable program or batch file.
The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

Traceback (most recent call last):
  File "C:\Users\Sleo\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Sleo\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Sleo\anaconda3\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "C:\Users\Sleo\anaconda3\lib\site-packages\kaggle\__init__.py", line 23, in <module>
    api.authenticate()
  File "C:\Users\Sleo\anaconda3\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\Sleo\.kaggle. Or use the environment method.


In [4]:
# unzipping

import zipfile
ref=zipfile.ZipFile('/content/face-mask-12k-images-dataset.zip','r')
ref.extractall('/content')
ref.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/face-mask-12k-images-dataset.zip'

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import matplotlib.pyplot as plt
import cv2
from PIL import Image

In [ ]:
x="/content/Face Mask Dataset/Train/WithMask/1212.png"
img=Image.open(x)
img = img.resize((300,300))
sr = np.array(img) 
plt.imshow(img)
plt.show()

In [ ]:
x="/content/Face Mask Dataset/Train/WithMask/1057.png"
img=Image.open(x)
img = img.resize((300,300))
sr = np.array(img) 
plt.imshow(img)
plt.show()

In [ ]:
x="/content/Face Mask Dataset/Train/WithoutMask/1022.png"
img=Image.open(x)
img = img.resize((300,300))
sr = np.array(img) 
plt.imshow(img)
plt.show()

In [ ]:
x="/content/Face Mask Dataset/Train/WithoutMask/1112.png"
img=Image.open(x)
img = img.resize((300,300))
sr = np.array(img) 
plt.imshow(img)
plt.show()

In [ ]:
 # Normalization 

from keras.preprocessing import image
# path set korchi
train_path = "/content/Face Mask Dataset/Train"
# image generation object
data=image.ImageDataGenerator(rescale=1./255)
#dataset normalization
x=data.flow_from_directory(train_path, 
                           target_size = (26,26),
                           batch_size = 32, 
                           class_mode = 'categorical')

In [ ]:
# Displaying the Classes

classes=['WithMask','WithoutMask']

In [ ]:
# Storing The Data and the Labels

data = []
labels = []
for i in classes:
    path = os.path.join(train_path,str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image=Image.open(path + '/' + a)
            image=image.resize((30,30))
            image=np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("error loading image.")
data = np.array(data)
labels = np.array(labels)
print(data.shape)
print(labels.shape)

In [ ]:
# splitting the training and testing data

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(data,labels,test_size=0.2,random_state=50)

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
# Lets Convert the labels to categorical

from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder # first amader oi string type category guloke int e ante hobe 
label_encoder = LabelEncoder()
y_train=np.array(y_train)
y_test=np.array(y_test)
y_train=label_encoder.fit_transform(y_train)
y_test=label_encoder.fit_transform(y_test)

In [ ]:
y_train=to_categorical(y_train,num_classes=2)
y_test=to_categorical(y_test,num_classes=2)

In [ ]:
# importing librareis for CNN

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout

In [ ]:
model= Sequential()

#Layer1: 32 Filters
model.add(Conv2D(32,kernel_size=(5,5),padding='valid',activation='relu',input_shape=x_train.shape[1:])) # convolution layer
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid')) #pooling layer
model.add(Dropout(rate=0.25)) # dropout layer
#Layer2: 64 Filters
model.add(Conv2D(64,kernel_size=(5,5),padding='valid',activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Dropout(rate=0.25))

#Flatten Layer
model.add(Flatten())

#Now we will add 1 fully connected layers
model.add(Dense(256,activation='relu'))

#Final Dropout Layer
model.add(Dropout(rate=0.5))

#Final Dense layer with 2 as Output is 2 Class
model.add(Dense(2, activation='softmax'))

In [ ]:
#Summary of the Model

model.summary()

In [ ]:
#compiling the model

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fitting
history=model.fit(x_train,y_train,batch_size=32,epochs=25,validation_data=(x_test, y_test))

In [ ]:
# accuracy of the model

score = model.evaluate(x_test,y_test,verbose = 0)
print('test accuracy: ', score[1]*100,'%')

to test our model

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def check(img):
    data=[]
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
    x=np.array(data)
    y=np.argmax(model.predict(x),axis=1)
    return image,y

In [ ]:
plot,prediction=check(r'/content/Face Mask Dataset/Train/WithoutMask/1067.png') # ans is without Mask
s=[str(i)for i in prediction] 
a="".join(s)
a=int(a)
print("Prediction: ", classes[a])
plt.imshow(plot)
plt.show()

In [ ]:
plot,prediction=check(r'/content/Face Mask Dataset/Train/WithMask/1153.png') # ans is with mask
s=[str(i)for i in prediction] 
a="".join(s)
a=int(a)
print("Prediction: ", classes[a])
plt.imshow(plot)
plt.show()

In [ ]:
plot,prediction=check(r'/content/Face Mask Dataset/Train/WithMask/1415.png') # ans is with mask 
s=[str(i)for i in prediction] 
a="".join(s)
a=int(a)
print("Prediction: ", classes[a])
plt.imshow(plot)
plt.show()

In [ ]:
plot,prediction=check(r'/content/Face Mask Dataset/Train/WithoutMask/1104.png') # ans is Without mask
s=[str(i)for i in prediction] 
a="".join(s)
a=int(a)
print("Prediction: ", classes[a])
plt.imshow(plot)
plt.show()

In [ ]:
import pickle
pickle.dump(model,open('maskdetector.pkl','wb'))

In [ ]:
model.save('model.h5')